# Example: Polycystic Kidney Disease use case

In [1]:
# Import modules
import os
import pickle

import pandas as pd
from dotenv import load_dotenv

from pyBiodatafuse import human_homologs, id_mapper
from pyBiodatafuse.annotators import (
    bgee,
    disgenet,
    kegg,
    minerva,
    molmedb,
    opentargets,
    pubchem,
    stringdb,
    wikipathways,
)
from pyBiodatafuse.constants import (
    BGEE_GENE_EXPRESSION_LEVELS_COL,
    DISGENET_DISEASE_COL,
    ENSEMBL_HOMOLOG_COL,
    KEGG_COL,
    OPENTARGETS_DISEASE_COMPOUND_COL,
    OPENTARGETS_GENE_COMPOUND_COL,
    OPENTARGETS_GO_COL,
    OPENTARGETS_REACTOME_COL,
    STRING_PPI_COL,
    WIKIPATHWAYS,
)
from pyBiodatafuse.graph import generator
from pyBiodatafuse.utils import combine_sources, combine_with_homologs, create_harmonized_input_file

# 1. Entity resolution using BridgeDB

### 1.1. Load the input list and convert it to a dataframe

Small set of mouse data used for debugging.

In [2]:
# TEST Mice Ensembl
genes_of_interest = """ENSMUSG00000067274
ENSMUSG00000000001
ENSMUSG00000084349
ENSMUSG00000025428
ENSMUSG00000044533"""

# TEST Mice use case =
genes_of_interest = """ENSMUSG00000026295
ENSMUSG00000022877
ENSMUSG00000020914
ENSMUSG00000024747
ENSMUSG00000032081
ENSMUSG00000004035
ENSMUSG00000072949
ENSMUSG00000028970
ENSMUSG00000028937
ENSMUSG00000075044"""

# TEST Rat Ensembl
# genes_of_interest = """ENSRNOG00060027926
# ENSRNOG00055005387
# ENSRNOG00060018596
# ENSRNOG00060011358
# ENSRNOG00055009275
# """

# TEST Human HGNC
# genes_of_interest = """CHRNG
# DMD
# AHR
# SCN4A
# LC25A1
# HTR3A"""

# TEST Human Ensembl
# genes_of_interest = """ENSG00000072080
# ENSG00000113905
# ENSG00000131747
# ENSG00000165092
# ENSG00000110245
# ENSG00000213366
# ENSG00000184227
# ENSG00000085563
# ENSG00000097021
# ENSG00000149742"""

gene_list = genes_of_interest.split("\n")
len(gene_list)

10

Mouse use case

In [3]:
# from pyBiodatafuse import data_loader, id_mapper

# data_input = data_loader.create_df_from_dea("data/full_de_genes_treated_vs_untreated_plus_cpm_fc.csv")
# data_filtered = data_input[data_input['DE'].isin([1, -1])]
# features_filtered = data_filtered['identifier']

# gene_list = features_filtered.tolist()
# print(len(gene_list))

In [4]:
data_input = pd.DataFrame(gene_list, columns=["identifier"])
data_input.head(20)

,identifier
0,ENSMUSG00000026295
1,ENSMUSG00000022877
2,ENSMUSG00000020914
3,ENSMUSG00000024747
4,ENSMUSG00000032081
5,ENSMUSG00000004035
6,ENSMUSG00000072949
7,ENSMUSG00000028970
8,ENSMUSG00000028937
9,ENSMUSG00000075044


In [5]:
print(gene_list)

['ENSMUSG00000026295', 'ENSMUSG00000022877', 'ENSMUSG00000020914', 'ENSMUSG00000024747', 'ENSMUSG00000032081', 'ENSMUSG00000004035', 'ENSMUSG00000072949', 'ENSMUSG00000028970', 'ENSMUSG00000028937', 'ENSMUSG00000075044']


### 1.2. Query BridgeDB

In [6]:
# Mouse usecase
input_species = "Mouse"

bridgedb_df, bridgedb_metadata = id_mapper.bridgedb_xref(
    identifiers=data_input,
    input_species=input_species,
    input_datasource="Ensembl",
    output_datasource="All",
)

# TEST Human data
# bridgedb_df, bridgedb_metadata = id_mapper.bridgedb_xref(
#     identifiers=data_input,
#     input_species="Human",
#     input_datasource="Ensembl",
#     output_datasource="All",
# )

bridgedb_df.head(25)

,identifier,identifier.source,target,target.source
0,ENSMUSG00000026295,Ensembl,4434677,Affy
1,ENSMUSG00000026295,Ensembl,Q8K1I3,Uniprot-TrEMBL
2,ENSMUSG00000026295,Ensembl,A_51_P166152,Agilent
3,ENSMUSG00000026295,Ensembl,5314352,Affy
4,ENSMUSG00000026295,Ensembl,4638627,Affy
5,ENSMUSG00000026295,Ensembl,4315932,Affy
6,ENSMUSG00000026295,Ensembl,1418916_a_at,Affy
7,ENSMUSG00000026295,Ensembl,5031793,Affy
8,ENSMUSG00000026295,Ensembl,ENSMUSG00000026295,Ensembl
9,ENSMUSG00000026295,Ensembl,ILMN_2903540,Illumina


### 1.3 Homologs

In [7]:
ensembl_homologs_df, ensembl_metadata = human_homologs.get_homologs(bridgedb_df=bridgedb_df)
ensembl_homologs_df.head()

,identifier,identifier.source,target,target.source,Ensembl_homologs
0,ENSMUSG00000026295,Ensembl,ENSMUSG00000026295,Ensembl,[{'homolog': 'ENSG00000072080'}]
1,ENSMUSG00000022877,Ensembl,ENSMUSG00000022877,Ensembl,[{'homolog': 'ENSG00000113905'}]
2,ENSMUSG00000020914,Ensembl,ENSMUSG00000020914,Ensembl,[{'homolog': 'ENSG00000131747'}]
3,ENSMUSG00000024747,Ensembl,ENSMUSG00000024747,Ensembl,[{'homolog': 'ENSG00000165092'}]
4,ENSMUSG00000032081,Ensembl,ENSMUSG00000032081,Ensembl,[{'homolog': 'ENSG00000110245'}]


In [8]:
ensembl_homologs_df.head(20)

,identifier,identifier.source,target,target.source,Ensembl_homologs
0,ENSMUSG00000026295,Ensembl,ENSMUSG00000026295,Ensembl,[{'homolog': 'ENSG00000072080'}]
1,ENSMUSG00000022877,Ensembl,ENSMUSG00000022877,Ensembl,[{'homolog': 'ENSG00000113905'}]
2,ENSMUSG00000020914,Ensembl,ENSMUSG00000020914,Ensembl,[{'homolog': 'ENSG00000131747'}]
3,ENSMUSG00000024747,Ensembl,ENSMUSG00000024747,Ensembl,[{'homolog': 'ENSG00000165092'}]
4,ENSMUSG00000032081,Ensembl,ENSMUSG00000032081,Ensembl,[{'homolog': 'ENSG00000110245'}]
5,ENSMUSG00000004035,Ensembl,ENSMUSG00000004035,Ensembl,[{'homolog': 'ENSG00000213366'}]
6,ENSMUSG00000072949,Ensembl,ENSMUSG00000072949,Ensembl,[{'homolog': 'ENSG00000184227'}]
7,ENSMUSG00000028970,Ensembl,ENSMUSG00000028970,Ensembl,[{'homolog': 'ENSG00000085563'}]
8,ENSMUSG00000028937,Ensembl,ENSMUSG00000028937,Ensembl,[{'homolog': 'ENSG00000097021'}]
9,ENSMUSG00000075044,Ensembl,ENSMUSG00000075044,Ensembl,[{'homolog': 'ENSG00000149742'}]


In [9]:
homologs = (
    ensembl_homologs_df[ENSEMBL_HOMOLOG_COL]
    .apply(
        lambda x: (
            x[0]["homolog"] if isinstance(x, list) and len(x) > 0 and "homolog" in x[0] else None
        )
    )
    .dropna()
    .tolist()
)

print(homologs)

['ENSG00000072080', 'ENSG00000113905', 'ENSG00000131747', 'ENSG00000165092', 'ENSG00000110245', 'ENSG00000213366', 'ENSG00000184227', 'ENSG00000085563', 'ENSG00000097021', 'ENSG00000149742']


### 1.4 Query homologs

In [10]:
input_species = "Human"

data_input_hl = pd.DataFrame(homologs, columns=["identifier"])

bridgedb_df_hl, bridgedb_metadata_hl = id_mapper.bridgedb_xref(
    identifiers=data_input_hl,
    input_species="Human",
    input_datasource="Ensembl",
    output_datasource="All",
)

bridgedb_df.head(25)

,identifier,identifier.source,target,target.source
0,ENSMUSG00000026295,Ensembl,4434677,Affy
1,ENSMUSG00000026295,Ensembl,Q8K1I3,Uniprot-TrEMBL
2,ENSMUSG00000026295,Ensembl,A_51_P166152,Agilent
3,ENSMUSG00000026295,Ensembl,5314352,Affy
4,ENSMUSG00000026295,Ensembl,4638627,Affy
5,ENSMUSG00000026295,Ensembl,4315932,Affy
6,ENSMUSG00000026295,Ensembl,1418916_a_at,Affy
7,ENSMUSG00000026295,Ensembl,5031793,Affy
8,ENSMUSG00000026295,Ensembl,ENSMUSG00000026295,Ensembl
9,ENSMUSG00000026295,Ensembl,ILMN_2903540,Illumina


# 2. Step-by-step graph generation

### 2.1. Gene-Disease edges


In [11]:
load_dotenv("disgenet.env")

disgenet_api_key = os.getenv("DISGENET_API_KEY")

In [12]:
disgenet_df, disgenet_metadata = disgenet.get_gene_disease(
    api_key=disgenet_api_key, bridgedb_df=bridgedb_df_hl
)
disgenet_df.head()

Querying DisGeNET: 100%|██████████| 10/10 [00:05<00:00,  1.73it/s]


,identifier,identifier.source,target,target.source,DISGENET_diseases
0,ENSG00000072080,Ensembl,6694,NCBI Gene,"[{'disease_name': 'Retinal dystrophy', 'HPO': ..."
1,ENSG00000085563,Ensembl,5243,NCBI Gene,"[{'disease_name': 'Breast Neoplasms', 'HPO': '..."
2,ENSG00000097021,Ensembl,11332,NCBI Gene,"[{'disease_name': nan, 'HPO': nan, 'NCI': nan,..."
3,ENSG00000110245,Ensembl,345,NCBI Gene,"[{'disease_name': 'Coronary Disease', 'HPO': N..."
4,ENSG00000113905,Ensembl,3273,NCBI Gene,"[{'disease_name': 'Breast Neoplasms', 'HPO': '..."


### 2.2 Disease-Compound edges

In [13]:
# Prepare the input to use DISGENET output as seed for OpenTargets
disease_mapping_df = create_harmonized_input_file(disgenet_df, DISGENET_DISEASE_COL, "EFO", "UMLS")
disease_mapping_df.head()

,identifier,identifier.source,target,target.source
0,UMLS_C1458155,UMLS,EFO_0003869,EFO
1,UMLS_C0014544,UMLS,EFO_0000474,EFO
2,UMLS_C0007131,UMLS,EFO_0003060,EFO
3,UMLS_C2239176,UMLS,EFO_0000182,EFO
4,UMLS_C2239176,UMLS,EFO_0000762,EFO


In [15]:
(
    opentargets_disease_compound_df,
    opentargets_disease_compound_metadata,
) = opentargets.get_disease_compound_interactions(disease_mapping_df)
opentargets_disease_compound_df.head()

Mapping PubChem: 100%|██████████| 1039/1039 [03:45<00:00,  4.62it/s]
/home/javi/pyBiodatafuse-3/src/pyBiodatafuse/annotators/opentargets.py:1133: UserWarning: Not all values in column 'adverse_effect' have the correct type '<class 'list'>'.
  check_columns_against_constants(


,identifier,identifier.source,target,target.source,OpenTargets_disease_compounds
0,UMLS_C0001175,UMLS,EFO_0000765,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL704', 'drugbank_i..."
1,UMLS_C0002103,UMLS,EFO_0005854,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL1201353', 'drugba..."
2,UMLS_C0003873,UMLS,EFO_0000685,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL2103743', 'drugba..."
3,UMLS_C0003977,UMLS,EFO_0004259,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL1200733', 'drugba..."
4,UMLS_C0004936,UMLS,EFO_0000677,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL4105630', 'drugba..."


In [16]:
opentargets_disease_compound_df[OPENTARGETS_DISEASE_COMPOUND_COL][0]

[{'chembl_id': 'CHEMBL:CHEMBL704',
  'drugbank_id': 'DrugBank:DB00244',
  'compound_cid': 'pubchem.compound:4075',
  'compound_name': 'MESALAMINE',
  'clincal_trial_phase': 4.0,
  'is_approved': True,
  'relation': 'treats',
  'adverse_effect_count': 37.0,
  'adverse_effect': [{'name': 'colitis ulcerative'},
   {'name': 'haematochezia'},
   {'name': 'colitis'},
   {'name': 'frequent bowel movements'},
   {'name': "crohn's disease"},
   {'name': 'myocarditis'},
   {'name': 'macular degeneration'},
   {'name': 'condition aggravated'},
   {'name': 'abdominal pain'},
   {'name': 'diarrhoea haemorrhagic'},
   {'name': 'pancreatitis'},
   {'name': 'weight decreased'},
   {'name': 'pyrexia'},
   {'name': 'pancreatitis acute'},
   {'name': 'pericarditis'},
   {'name': 'tubulointerstitial nephritis'},
   {'name': 'diarrhoea'},
   {'name': 'drug intolerance'},
   {'name': 'rectal haemorrhage'},
   {'name': 'vitamin d deficiency'},
   {'name': 'impaired quality of life'},
   {'name': 'mucous stoo

### 2.3 Compound Annotation

#### Compounds from OpenTargets

In [17]:
opentargets_compound_df, opentargets_compound_metadata = opentargets.get_gene_compound_interactions(
    bridgedb_df=bridgedb_df_hl
)
opentargets_compound_df.head()

Mapping PubChem: 100%|██████████| 14/14 [00:05<00:00,  2.59it/s]
/home/javi/pyBiodatafuse-3/src/pyBiodatafuse/annotators/opentargets.py:607: UserWarning: Not all values in column 'adverse_effect_count' have the correct type '<class 'int'>'.
  check_columns_against_constants(


,identifier,identifier.source,target,target.source,OpenTargets_gene_compounds
0,ENSG00000072080,Ensembl,ENSG00000072080,Ensembl,"[{'chembl_id': nan, 'drugbank_id': nan, 'compo..."
1,ENSG00000085563,Ensembl,ENSG00000085563,Ensembl,"[{'chembl_id': 'CHEMBL:CHEMBL1086218', 'drugba..."
2,ENSG00000097021,Ensembl,ENSG00000097021,Ensembl,"[{'chembl_id': nan, 'drugbank_id': nan, 'compo..."
3,ENSG00000110245,Ensembl,ENSG00000110245,Ensembl,"[{'chembl_id': 'CHEMBL:CHEMBL3544989', 'drugba..."
4,ENSG00000113905,Ensembl,ENSG00000113905,Ensembl,"[{'chembl_id': nan, 'drugbank_id': nan, 'compo..."


In [18]:
opentargets_compound_df[OPENTARGETS_GENE_COMPOUND_COL][0]

[{'chembl_id': nan,
  'drugbank_id': nan,
  'compound_cid': nan,
  'compound_name': nan,
  'clincal_trial_phase': nan,
  'is_approved': nan,
  'relation': nan,
  'adverse_effect_count': nan,
  'adverse_effect': nan}]

#### Compounds from PubChem

In [19]:
pubchem_assay_df, pubchem_assay_metadata = pubchem.get_protein_compound_screened(
    bridgedb_df=bridgedb_df_hl
)

Querying PubChem: 100%|██████████| 2/2 [00:00<00:00,  6.36it/s]
/tmp/ipykernel_438775/2311319464.py:1: UserWarning: There is no annotation for your input list in PubChem.
  pubchem_assay_df, pubchem_assay_metadata = pubchem.get_protein_compound_screened(


### 2.4 Gene-Pathways edges

#### Pathways from WikiPathways

In [20]:
wikipathways_df, wikipathways_metadata = wikipathways.get_gene_wikipathways(bridgedb_df=bridgedb_df)
wikipathways_df.head()

Querying WikiPathways: 100%|██████████| 1/1 [00:24<00:00, 24.59s/it]


,identifier,identifier.source,target,target.source,WikiPathways
0,ENSMUSG00000004035,Ensembl,68312,NCBI Gene,"[{'pathway_id': 'WP:WP4466', 'pathway_label': ..."
1,ENSMUSG00000020914,Ensembl,21973,NCBI Gene,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."
2,ENSMUSG00000022877,Ensembl,94175,NCBI Gene,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."
3,ENSMUSG00000024747,Ensembl,26358,NCBI Gene,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."
4,ENSMUSG00000026295,Ensembl,75396,NCBI Gene,"[{'pathway_id': nan, 'pathway_label': nan, 'pa..."


In [21]:
wikipathways_df[WIKIPATHWAYS][0]

[{'pathway_id': 'WP:WP4466',
  'pathway_label': 'Oxidative stress and redox pathway',
  'pathway_gene_count': 92.0},
 {'pathway_id': 'WP:WP1251',
  'pathway_label': 'Metapathway biotransformation',
  'pathway_gene_count': 143.0},
 {'pathway_id': 'WP:WP164',
  'pathway_label': 'Glutathione metabolism',
  'pathway_gene_count': 19.0}]

#### Pathways from KEGG

In [22]:
kegg_df, kegg_metadata = kegg.get_pathways(bridgedb_df)
kegg_df.head()

,identifier,identifier.source,target,target.source,KEGG_pathways
0,ENSMUSG00000026295,Ensembl,75396,NCBI Gene,"[{'pathway_id': nan, 'pathway_label': nan, 'ge..."
1,ENSMUSG00000022877,Ensembl,94175,NCBI Gene,"[{'pathway_id': nan, 'pathway_label': nan, 'ge..."
2,ENSMUSG00000020914,Ensembl,21973,NCBI Gene,"[{'pathway_id': 'path:mmu01524', 'pathway_labe..."
3,ENSMUSG00000024747,Ensembl,26358,NCBI Gene,"[{'pathway_id': 'path:mmu00830', 'pathway_labe..."
4,ENSMUSG00000032081,Ensembl,11814,NCBI Gene,"[{'pathway_id': 'path:mmu03320', 'pathway_labe..."


In [23]:
kegg_df[KEGG_COL][3][0]

{'pathway_id': 'path:mmu00830',
 'pathway_label': 'Retinol metabolism - Mus musculus (house mouse)',
 'gene_count': 101,
 'compounds': [{'KEGG_identifier': 'C00376'},
  {'KEGG_identifier': 'C00473'},
  {'KEGG_identifier': 'C00777'},
  {'KEGG_identifier': 'C00778'},
  {'KEGG_identifier': 'C00899'},
  {'KEGG_identifier': 'C02075'},
  {'KEGG_identifier': 'C02094'},
  {'KEGG_identifier': 'C02110'},
  {'KEGG_identifier': 'C02588'},
  {'KEGG_identifier': 'C03455'},
  {'KEGG_identifier': 'C05914'},
  {'KEGG_identifier': 'C05915'},
  {'KEGG_identifier': 'C05916'},
  {'KEGG_identifier': 'C05917'},
  {'KEGG_identifier': 'C11061'},
  {'KEGG_identifier': 'C15492'},
  {'KEGG_identifier': 'C15493'},
  {'KEGG_identifier': 'C16677'},
  {'KEGG_identifier': 'C16678'},
  {'KEGG_identifier': 'C16679'},
  {'KEGG_identifier': 'C16680'},
  {'KEGG_identifier': 'C16681'},
  {'KEGG_identifier': 'C16682'},
  {'KEGG_identifier': 'C16683'},
  {'KEGG_identifier': 'C21797'}]}

#### Reactome pathways from OpenTargets

In [24]:
opentargets_reactome_df, opentargets_reactome_metadata = opentargets.get_gene_reactome_pathways(
    bridgedb_df=bridgedb_df_hl
)
opentargets_reactome_df.head()

Processing gene-pathway interactions: 100%|██████████| 10/10 [00:00<00:00, 410.61it/s]


,identifier,identifier.source,target,target.source,OpenTargets_reactome
0,ENSG00000072080,Ensembl,ENSG00000072080,Ensembl,"[{'pathway_label': 'Platelet degranulation ', ..."
1,ENSG00000085563,Ensembl,ENSG00000085563,Ensembl,[{'pathway_label': 'Abacavir transmembrane tra...
2,ENSG00000097021,Ensembl,ENSG00000097021,Ensembl,[{'pathway_label': 'Mitochondrial Fatty Acid B...
3,ENSG00000110245,Ensembl,ENSG00000110245,Ensembl,[{'pathway_label': 'Retinoid metabolism and tr...
4,ENSG00000113905,Ensembl,ENSG00000113905,Ensembl,"[{'pathway_label': 'Platelet degranulation ', ..."


In [25]:
opentargets_reactome_df[OPENTARGETS_REACTOME_COL][0]

[{'pathway_label': 'Platelet degranulation ',
  'pathway_id': 'Reactome:R-HSA-114608'},
 {'pathway_label': 'Regulation of Insulin-like Growth Factor (IGF) transport and uptake by Insulin-like Growth Factor Binding Proteins (IGFBPs)',
  'pathway_id': 'Reactome:R-HSA-381426'},
 {'pathway_label': 'Post-translational protein phosphorylation',
  'pathway_id': 'Reactome:R-HSA-8957275'}]

### 2.5 Gene Ontology from OpenTargets

In [26]:
opentargets_go_df, opentargets_go_metadata = opentargets.get_gene_go_process(
    bridgedb_df=bridgedb_df_hl
)
opentargets_go_df.head()

Processing gene annotation: 100%|██████████| 10/10 [00:00<00:00, 275.63it/s]


,identifier,identifier.source,target,target.source,OpenTargets_go
0,ENSG00000072080,Ensembl,ENSG00000072080,Ensembl,"[{'go_id': 'GO:0005788', 'go_name': 'endoplasm..."
1,ENSG00000085563,Ensembl,ENSG00000085563,Ensembl,"[{'go_id': 'GO:0008559', 'go_name': 'ABC-type ..."
2,ENSG00000097021,Ensembl,ENSG00000097021,Ensembl,"[{'go_id': 'GO:0070062', 'go_name': 'extracell..."
3,ENSG00000110245,Ensembl,ENSG00000110245,Ensembl,"[{'go_id': 'GO:0005543', 'go_name': 'phospholi..."
4,ENSG00000113905,Ensembl,ENSG00000113905,Ensembl,"[{'go_id': 'GO:0005576', 'go_name': 'extracell..."


In [27]:
opentargets_go_df[OPENTARGETS_GO_COL][0]

[{'go_id': 'GO:0005788',
  'go_name': 'endoplasmic reticulum lumen',
  'go_type': 'C'},
 {'go_id': 'GO:0031089',
  'go_name': 'platelet dense granule lumen',
  'go_type': 'C'},
 {'go_id': 'GO:0004866',
  'go_name': 'endopeptidase inhibitor activity',
  'go_type': 'F'},
 {'go_id': 'GO:0046849', 'go_name': 'bone remodeling', 'go_type': 'P'},
 {'go_id': 'GO:0062023',
  'go_name': 'collagen-containing extracellular matrix',
  'go_type': 'C'},
 {'go_id': 'GO:0005576', 'go_name': 'extracellular region', 'go_type': 'C'},
 {'go_id': 'GO:0001501',
  'go_name': 'skeletal system development',
  'go_type': 'P'}]

### 2.6. Protein-Protein Interactions

In [28]:
input_species = "Mouse"
ppi_df, ppi_metadata = stringdb.get_ppi(bridgedb_df=bridgedb_df, species=input_species)
ppi_df.head()

,identifier,identifier.source,target,target.source,StringDB_ppi
0,ENSMUSG00000026295,Ensembl,ENSMUSG00000026295,Ensembl,"[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc..."
1,ENSMUSG00000022877,Ensembl,ENSMUSG00000022877,Ensembl,"[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc..."
2,ENSMUSG00000020914,Ensembl,ENSMUSG00000020914,Ensembl,"[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc..."
3,ENSMUSG00000024747,Ensembl,ENSMUSG00000024747,Ensembl,"[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc..."
4,ENSMUSG00000032081,Ensembl,ENSMUSG00000032081,Ensembl,"[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc..."


In [29]:
ppi_df[STRING_PPI_COL][3]

[{'stringdb_link_to': nan,
  'Ensembl': nan,
  'score': nan,
  'Uniprot-TrEMBL': None,
  'Uniprot-TrEMBL_link': None}]

### 2.7 Gene expression edges

In [30]:
bgee_df, bgee_metadata = bgee.get_gene_expression(bridgedb_df=bridgedb_df)
bgee_df.head()

,identifier,identifier.source,target,target.source,Bgee_gene_expression_levels
0,ENSMUSG00000004035,Ensembl,ENSMUSG00000004035,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000955', 'a..."
1,ENSMUSG00000020914,Ensembl,ENSMUSG00000020914,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a..."
2,ENSMUSG00000022877,Ensembl,ENSMUSG00000022877,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000955', 'a..."
3,ENSMUSG00000024747,Ensembl,ENSMUSG00000024747,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a..."
4,ENSMUSG00000026295,Ensembl,ENSMUSG00000026295,Ensembl,"[{'anatomical_entity_id': 'UBERON_0002371', 'a..."


In [31]:
bgee_df[BGEE_GENE_EXPRESSION_LEVELS_COL][1]

[{'anatomical_entity_id': 'UBERON_0000178',
  'anatomical_entity_name': 'blood',
  'expression_level': 65.57567,
  'confidence_level_id': 'CIO_0000029',
  'confidence_level_name': 'high confidence level',
  'developmental_stage_id': 'UBERON_0000104',
  'developmental_stage_name': 'life cycle'},
 {'anatomical_entity_id': 'UBERON_0002371',
  'anatomical_entity_name': 'bone marrow',
  'expression_level': 91.47872,
  'confidence_level_id': 'CIO_0000029',
  'confidence_level_name': 'high confidence level',
  'developmental_stage_id': 'UBERON_0000104',
  'developmental_stage_name': 'life cycle'},
 {'anatomical_entity_id': 'UBERON_0000955',
  'anatomical_entity_name': 'brain',
  'expression_level': 51.9632,
  'confidence_level_id': 'CIO_0000029',
  'confidence_level_name': 'high confidence level',
  'developmental_stage_id': 'UBERON_0000104',
  'developmental_stage_name': 'life cycle'},
 {'anatomical_entity_id': 'UBERON_0000310',
  'anatomical_entity_name': 'breast',
  'expression_level': 62.

### 2.8 Transporter Inhibitors

In [32]:
inhibitor_df, inhibitor_metadata = molmedb.get_gene_compound_inhibitor(bridgedb_df=bridgedb_df_hl)
inhibitor_df.head()

""


# 3. Generating Graph

### 3.1 Combing all the results into single dataframe

In [33]:
#        bgee_df,
#        disgenet_df,
#        minerva_df,
#        opentargets_reactome_df,
#        opentargets_go_df,
#        opentargets_compound_df,
#        inhibitor_df,
#        kegg_df,

combined_df = combine_sources(
    bridgedb_df,
    [wikipathways_df, ppi_df, ensembl_homologs_df, kegg_df],
)

combined_df = combine_with_homologs(
    combined_df,
    [
        opentargets_reactome_df,
        opentargets_go_df,
        opentargets_compound_df,
        disgenet_df,
        inhibitor_df,
    ],
)


combined_df.head(10)

,identifier,identifier.source,target,target.source,WikiPathways,StringDB_ppi,Ensembl_homologs,KEGG_pathways,homolog,OpenTargets_reactome,OpenTargets_go,OpenTargets_gene_compounds,DISGENET_diseases
0,ENSMUSG00000026295,Ensembl,ENSMUSG00000026295,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000072080'}],"[{'pathway_id': nan, 'pathway_label': nan, 'ge...",ENSG00000072080,"[{'pathway_label': 'Platelet degranulation ', ...","[{'go_id': 'GO:0005788', 'go_name': 'endoplasm...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Retinal dystrophy', 'HPO': ..."
1,ENSMUSG00000022877,Ensembl,ENSMUSG00000022877,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000113905'}],"[{'pathway_id': nan, 'pathway_label': nan, 'ge...",ENSG00000113905,"[{'pathway_label': 'Platelet degranulation ', ...","[{'go_id': 'GO:0005576', 'go_name': 'extracell...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Breast Neoplasms', 'HPO': '..."
2,ENSMUSG00000020914,Ensembl,ENSMUSG00000020914,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000131747'}],"[{'pathway_id': 'path:mmu01524', 'pathway_labe...",ENSG00000131747,[{'pathway_label': 'SUMOylation of DNA replica...,"[{'go_id': 'GO:0046982', 'go_name': 'protein h...","[{'chembl_id': 'CHEMBL:CHEMBL44657', 'drugbank...","[{'disease_name': 'Breast Neoplasms', 'HPO': '..."
3,ENSMUSG00000024747,Ensembl,ENSMUSG00000024747,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000165092'}],"[{'pathway_id': 'path:mmu00830', 'pathway_labe...",ENSG00000165092,"[{'pathway_label': 'Fructose catabolism', 'pat...","[{'go_id': 'GO:0051287', 'go_name': 'NAD bindi...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Melanoma', 'HPO': 'HPO:HP:0..."
4,ENSMUSG00000032081,Ensembl,ENSMUSG00000032081,Ensembl,"[{'pathway_id': 'WP:WP1', 'pathway_label': 'St...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000110245'}],"[{'pathway_id': 'path:mmu03320', 'pathway_labe...",ENSG00000110245,[{'pathway_label': 'Retinoid metabolism and tr...,"[{'go_id': 'GO:0005543', 'go_name': 'phospholi...","[{'chembl_id': 'CHEMBL:CHEMBL3544989', 'drugba...","[{'disease_name': 'Coronary Disease', 'HPO': N..."
5,ENSMUSG00000004035,Ensembl,ENSMUSG00000004035,Ensembl,"[{'pathway_id': 'WP:WP4466', 'pathway_label': ...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000213366'}],"[{'pathway_id': 'path:mmu00480', 'pathway_labe...",ENSG00000213366,"[{'pathway_label': 'Glutathione conjugation', ...","[{'go_id': 'GO:0043295', 'go_name': 'glutathio...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Carcinoma, Non Small Cell L..."
6,ENSMUSG00000072949,Ensembl,ENSMUSG00000072949,Ensembl,"[{'pathway_id': 'WP:WP4346', 'pathway_label': ...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000184227'}],"[{'pathway_id': 'path:mmu00062', 'pathway_labe...",ENSG00000184227,[{'pathway_label': 'Mitochondrial Fatty Acid B...,"[{'go_id': 'GO:0052689', 'go_name': 'carboxyli...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Hepatomegaly', 'HPO': 'HPO:..."
7,ENSMUSG00000028970,Ensembl,ENSMUSG00000028970,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000085563'}],"[{'pathway_id': 'path:mmu02010', 'pathway_labe...",ENSG00000085563,[{'pathway_label': 'Abacavir transmembrane tra...,"[{'go_id': 'GO:0008559', 'go_name': 'ABC-type ...","[{'chembl_id': 'CHEMBL:CHEMBL1086218', 'drugba...","[{'disease_name': 'Breast Neoplasms', 'HPO': '..."
8,ENSMUSG00000028937,Ensembl,ENSMUSG00000028937,Ensembl,"[{

In [34]:
combined_df.shape

(10, 13)

### 3.2 Exporting the database in pickle format

In [35]:
with open("combined_df.pkl", "wb") as out:
    pickle.dump(combined_df, out)
# with open("opentargets_disease_compound_df.pkl", "wb") as out:
#     pickle.dump(opentargets_disease_compound_df, out)

### 3.3 Creating a graph from the annotated dataframe

In [36]:
# combined_df = generator.load_dataframe_from_pickle("combined_df.pkl")
# opentargets_disease_compound_df = generator.load_dataframe_from_pickle(
#     "opentargets_disease_compound_df.pkl"
# )

combined_df.head(15)

,identifier,identifier.source,target,target.source,WikiPathways,StringDB_ppi,Ensembl_homologs,KEGG_pathways,homolog,OpenTargets_reactome,OpenTargets_go,OpenTargets_gene_compounds,DISGENET_diseases
0,ENSMUSG00000026295,Ensembl,ENSMUSG00000026295,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000072080'}],"[{'pathway_id': nan, 'pathway_label': nan, 'ge...",ENSG00000072080,"[{'pathway_label': 'Platelet degranulation ', ...","[{'go_id': 'GO:0005788', 'go_name': 'endoplasm...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Retinal dystrophy', 'HPO': ..."
1,ENSMUSG00000022877,Ensembl,ENSMUSG00000022877,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000113905'}],"[{'pathway_id': nan, 'pathway_label': nan, 'ge...",ENSG00000113905,"[{'pathway_label': 'Platelet degranulation ', ...","[{'go_id': 'GO:0005576', 'go_name': 'extracell...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Breast Neoplasms', 'HPO': '..."
2,ENSMUSG00000020914,Ensembl,ENSMUSG00000020914,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000131747'}],"[{'pathway_id': 'path:mmu01524', 'pathway_labe...",ENSG00000131747,[{'pathway_label': 'SUMOylation of DNA replica...,"[{'go_id': 'GO:0046982', 'go_name': 'protein h...","[{'chembl_id': 'CHEMBL:CHEMBL44657', 'drugbank...","[{'disease_name': 'Breast Neoplasms', 'HPO': '..."
3,ENSMUSG00000024747,Ensembl,ENSMUSG00000024747,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000165092'}],"[{'pathway_id': 'path:mmu00830', 'pathway_labe...",ENSG00000165092,"[{'pathway_label': 'Fructose catabolism', 'pat...","[{'go_id': 'GO:0051287', 'go_name': 'NAD bindi...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Melanoma', 'HPO': 'HPO:HP:0..."
4,ENSMUSG00000032081,Ensembl,ENSMUSG00000032081,Ensembl,"[{'pathway_id': 'WP:WP1', 'pathway_label': 'St...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000110245'}],"[{'pathway_id': 'path:mmu03320', 'pathway_labe...",ENSG00000110245,[{'pathway_label': 'Retinoid metabolism and tr...,"[{'go_id': 'GO:0005543', 'go_name': 'phospholi...","[{'chembl_id': 'CHEMBL:CHEMBL3544989', 'drugba...","[{'disease_name': 'Coronary Disease', 'HPO': N..."
5,ENSMUSG00000004035,Ensembl,ENSMUSG00000004035,Ensembl,"[{'pathway_id': 'WP:WP4466', 'pathway_label': ...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000213366'}],"[{'pathway_id': 'path:mmu00480', 'pathway_labe...",ENSG00000213366,"[{'pathway_label': 'Glutathione conjugation', ...","[{'go_id': 'GO:0043295', 'go_name': 'glutathio...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Carcinoma, Non Small Cell L..."
6,ENSMUSG00000072949,Ensembl,ENSMUSG00000072949,Ensembl,"[{'pathway_id': 'WP:WP4346', 'pathway_label': ...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000184227'}],"[{'pathway_id': 'path:mmu00062', 'pathway_labe...",ENSG00000184227,[{'pathway_label': 'Mitochondrial Fatty Acid B...,"[{'go_id': 'GO:0052689', 'go_name': 'carboxyli...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'disease_name': 'Hepatomegaly', 'HPO': 'HPO:..."
7,ENSMUSG00000028970,Ensembl,ENSMUSG00000028970,Ensembl,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc...",[{'homolog': 'ENSG00000085563'}],"[{'pathway_id': 'path:mmu02010', 'pathway_labe...",ENSG00000085563,[{'pathway_label': 'Abacavir transmembrane tra...,"[{'go_id': 'GO:0008559', 'go_name': 'ABC-type ...","[{'chembl_id': 'CHEMBL:CHEMBL1086218', 'drugba...","[{'disease_name': 'Breast Neoplasms', 'HPO': '..."
8,ENSMUSG00000028937,Ensembl,ENSMUSG00000028937,Ensembl,"[{

In [37]:
combined_df["KEGG_pathways"][2]

[{'pathway_id': 'path:mmu01524',
  'pathway_label': 'Platinum drug resistance - Mus musculus (house mouse)',
  'gene_count': 0,
  'compounds': [{'KEGG_identifier': None}]},
 {'gene_count': 0, 'compounds': [{'KEGG_identifier': None}]}]

In [38]:
opentargets_disease_compound_df.head()

,identifier,identifier.source,target,target.source,OpenTargets_disease_compounds
0,UMLS_C0001175,UMLS,EFO_0000765,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL704', 'drugbank_i..."
1,UMLS_C0002103,UMLS,EFO_0005854,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL1201353', 'drugba..."
2,UMLS_C0003873,UMLS,EFO_0000685,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL2103743', 'drugba..."
3,UMLS_C0003977,UMLS,EFO_0004259,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL1200733', 'drugba..."
4,UMLS_C0004936,UMLS,EFO_0000677,EFO,"[{'chembl_id': 'CHEMBL:CHEMBL4105630', 'drugba..."


In [39]:
pygraph = generator.build_networkx_graph(
    combined_df, homolog_df_list=[opentargets_compound_df, disgenet_df]
)

100%|██████████| 10/10 [00:00<00:00, 1787.70it/s]


In [40]:
print(pygraph)

MultiDiGraph with 446 nodes and 493 edges


### 3.4 Store the graph

In [41]:
with open("networkx_graph_test.pkl", "wb") as out:
    pickle.dump(pygraph, out)

### 3.5 Visualize the graph

In [ ]:
# pos = nx.circular_layout(pygraph)

# plt.figure(3, figsize=(30, 30))
# nx.draw(pygraph, pos)
# plt.show()

# 4. Exporting Graph to external sources

### 4.1 Cytoscape
Make sure that the Cytoscape is open

In [43]:
from pyBiodatafuse.graph import cytoscape

cytoscape.load_graph(pygraph, network_name="Test network")

Applying default style...
Applying preferred layout


### 4.2 Neo4j

In [44]:
from pyBiodatafuse.graph import neo4j

neo4j.save_graph_to_graphml(pygraph, "networkx_graph_usecase.graphml")

##### Steps to load the graph in Neo4j

- Add `.graphml` file in **import** subfolder of the DBMS folder
- Install apoc plugin
- Create `apoc.conf` file:
    ```
    apoc.trigger.enabled=true
    apoc.import.file.enabled=true
    apoc.export.file.enabled=true
    apoc.import.file.use_neo4j_config=true
    ```
- Add `apoc.conf` file to **conf** subfolder of the DBMS folder
- Open Neo4j Browser
- (Optionl, only run if you have imported a graph  before) Remove all the nodes before importing `.graphml` file

    ```MATCH (n) DETACH DELETE n```

- Import `.graphml` file

    ```call apoc.import.graphml('file:///networkx_graph_test.graphml',{readLabels:TRUE})```

- Add indexes after importing the graph for improving the performance of queries

    ```
    create index Gene for (n:Gene) on (n.node_type)
    create index Pathway for (n:Pathway) on (n.node_type)
    create index `Biological Process` for (n:`Biological Process`) on (n.node_type)
    create index `Molecular Function` for (n:`Molecular Function`) on (n.node_type)
    create index `Cellular Component` for (n:`Cellular Component`) on (n.node_type)
    create index Disease for (n:Disease) on (n.node_type)
    create index Compound for (n:Compound) on (n.node_type)
    create index `Side Effect` for (n:`Side Effect`) on (n.node_type)
    ```
    

- Count the number of each node type
    - total (```MATCH (n) RETURN count(n)```) 
        - Gene (```MATCH (n:Gene) RETURN count(n)```)
        - Pathway (```MATCH (n:Pathway) RETURN count(n)```)
            - WikiPathways (```MATCH (n:Pathway {source: "WikiPathways"}) RETURN count(n)```) 
            - OpenTargets, Reactome (```MATCH (n:Pathway {source: "OpenTargets"}) RETURN count(n)```) 
            - MINERVA (```MATCH (n:Pathway {source: "MINERVA"}) RETURN count(n)```) 
        - Biological Process (```MATCH (n:`Biological Process`) RETURN count(n)```) 
        - Molecular Function (```MATCH (n:`Molecular Function`) RETURN count(n)```) 
        - Cellular Component (```MATCH (n:`Cellular Component`) RETURN count(n)```) 
        - Disease (```MATCH (n:Disease) RETURN count(n)```) 
        - Compound (```MATCH (n:Compound) RETURN count(n)```)
        - Side Effect (```MATCH (n:`Side Effect`) RETURN count(n)```) 
- Count the number of each edge type
    - total (```MATCH ()-[r]->() RETURN count(r)```) 
        - interacts_with (```MATCH ()-[r:interacts_with]->() RETURN count(r)```) 
        - part_of (```MATCH ()-[r:part_of]->() RETURN count(r)```) 
            - WikiPathways (```MATCH ()-[r:part_of {source: "WikiPathways"}]->() RETURN count(r)```) 
            - OpenTargets, Reactome (```MATCH ()-[r:part_of {source: "OpenTargets"}]->() RETURN count(r)```) 
            - MINERVA (```MATCH ()-[r:part_of {source: "MINERVA"}]->() RETURN count(r)```) 
        - activates (```MATCH ()-[r:activates]->() RETURN count(r)```) 
        - treats (```MATCH ()-[r:treats]->() RETURN count(r)```) 
        - has_side_effect (```MATCH ()-[r:has_side_effect]->() RETURN count(r)```) 
        - inhibits (```MATCH ()-[r:inhibits]->() RETURN count(r)```) = 71
        - associated_with (```MATCH ()-[r:associated_with]->() RETURN count(r)```) 

- Export the graph as a `.csv` file

    ```call apoc.export.csv.all("networkx_graph_test.csv",{})```